<a href="https://colab.research.google.com/github/poolGolez/tf-playground/blob/main/exercises/%5Bregression%5D_habitability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Habitability

# Habitability

📗 **Dataset Source**: [Kaggle](https://www.kaggle.com/datasets/ifteshanajnin/get-a-room-ml-hackathon-by-hackerearth)

## Ingest

In [5]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/gdrive')

Mounted at /gdrive


In [6]:
FILE_LOCATION = '/gdrive/My Drive/Me/Learning/TensorFlow Developer Certificate: Zero to Mastery/data/habitability/train.csv'

df = pd.read_csv(FILE_LOCATION)
# print("Dataset")
# print(f"Number of rows: {len(df)}")
# print(f"Number of columns:{len(df.columns)}")
# print(f"Columns: {np.array(df.columns)}")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39499 entries, 0 to 39498
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Property_ID             39499 non-null  object 
 1   Property_Type           39499 non-null  object 
 2   Property_Area           39499 non-null  int64  
 3   Number_of_Windows       37845 non-null  float64
 4   Number_of_Doors         39499 non-null  int64  
 5   Furnishing              38457 non-null  object 
 6   Frequency_of_Powercuts  38116 non-null  float64
 7   Power_Backup            39499 non-null  object 
 8   Water_Supply            39499 non-null  object 
 9   Traffic_Density_Score   39499 non-null  float64
 10  Crime_Rate              38712 non-null  object 
 11  Dust_and_Noise          38280 non-null  object 
 12  Air_Quality_Index       39499 non-null  float64
 13  Neighborhood_Review     39499 non-null  float64
 14  Habitability_score      39499 non-null

## Exploratory Data Analysis

In [7]:
df['Property_Type'].unique()

array(['Apartment', 'Bungalow', 'Single-family home', 'Duplex',
       '#R%$G&867', 'Container Home'], dtype=object)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39499 entries, 0 to 39498
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Property_ID             39499 non-null  object 
 1   Property_Type           39499 non-null  object 
 2   Property_Area           39499 non-null  int64  
 3   Number_of_Windows       37845 non-null  float64
 4   Number_of_Doors         39499 non-null  int64  
 5   Furnishing              38457 non-null  object 
 6   Frequency_of_Powercuts  38116 non-null  float64
 7   Power_Backup            39499 non-null  object 
 8   Water_Supply            39499 non-null  object 
 9   Traffic_Density_Score   39499 non-null  float64
 10  Crime_Rate              38712 non-null  object 
 11  Dust_and_Noise          38280 non-null  object 
 12  Air_Quality_Index       39499 non-null  float64
 13  Neighborhood_Review     39499 non-null  float64
 14  Habitability_score      39499 non-null